### Train BERT Tokenizer
* https://www.thepythoncode.com/article/pretraining-bert-huggingface-transformers-in-python
* https://huggingface.co/transformers/v3.2.0/training.html
* 토크나이저 옵션을 바꿔서 다시 사전을 생성하고 pre-train 시도(2022.09.20)
    * 한글의 경우 : do_lower_case=False, strip_accents=False
    * 이렇게 안하면 자소단위로 쪼개지는 걸로 알고 있어요
    * tokenizer = BertTokenizer(os.path.join(MODEL_PATH, "kopat-vocab.txt"), do_lower_case=False, strip_accents=False)
    * https://keep-steady.tistory.com/37

In [1]:
import os
import json
import re
import unicodedata
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from transformers import *
from tokenizers import *
from datasets import *
from sklearn.model_selection import train_test_split
import nltk
from nltk.data import load

/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pre-train data 생성

In [2]:
try:
#     train_data_df = pd.read_csv('data/c09k_170k_pre_train.txt', sep='\t')  # text와 라벨 파일
#     test_dataset_df = pd.read_csv('data/c09k_170k_pre_test.txt', sep='\t')
#     train_dataset = Dataset.from_pandas(train_data_df)  # Dataset 객체 생성
#     test_dataset = Dataset.from_pandas(test_dataset_df)
#     d = DatasetDict()  # DatasetDict 객체 생성
#     d['train'] = train_dataset
#     d['test'] = test_dataset
    with open('c09k_170k_pre_train.pkl', 'rb') as f:
        d = pickle.load(f)
#     d
    print('Load completed')
except:
    dataset = Dataset.from_text('data/c09k_170k_corpus.txt')
    d = dataset.train_test_split(test_size=0.075)
    def dataset_to_text(dataset, output_filename="data.txt"):
        """Utility function to save dataset text to disk,
        useful for using the texts to train the tokenizer 
        (as the tokenizer accepts files)"""
        with open(output_filename, "w") as f:
            for t in dataset["text"]:
                print(t, file=f)
#     dataset_to_text(d["train"], "data/c09k_170k_pre_train.txt")
#     dataset_to_text(d["test"], "data/c09k_170k_pre_test.txt")
    with open('c09k_170k_pre_train.pkl', 'wb') as f:
        pickle.dump(d, f)
    print('Load corpus and split completed')

Load completed


In [3]:
d["train"], d["test"]
for t in d["train"]["text"][:3]:
    print(t)
    print("="*50)

일반식(8-0)에서 나타나는 화합물이, 다음의 일반식(8-1), (8-2), (8-3), (8-4), (8-5), (8-6) 및(8-7)으로 나타나는 특허 청구의 범위 제 8항에 기재된 광학 활성 나프타렌 유도체.10.
하기 일반식(1)로 표시되는 화합물인 것을 특징으로 하는 유기 일렉트로루미네센스 소자 재료.
1.발명은 순수한 유기 분자, 및 유기 발광 다이오드(OLED) 및 그 밖에 다른 유기 광전자 디바이스에서의 이의 용도에 관한 것이다.2.발명의 일 구현예는 최저 여기된 단일항 상태(S1)와 그것보다 낮은 삼중항 상태(T1) 사이의 △E(S1-T1) 값이 5000 cm-1 이하, 특히 3000 cm-1 이하, 1500 cm-1 이하 또는 1000 cm-1 이하이며, 및/또는 방출 수명이 150 μs 이하, 특히 100 μs 이하, 50 μs 이하 또는 10 μs 이하이며, 및/또는 주요 방출 밴드(main emission band)의 반-높이 너비(half-height width)가 0.55 eV 미만, 특히 0.50 eV 미만, 0.48 eV 미만 또는 0.45 eV 미만인 유기 분자에 관한 것이다.3.발명은 [0273] (a) 본 발명에 따른 하나 이상의 유기 분자, 특히 이미터 및/또는 호스트로서의 유기 분자, 및 [0274] (b) 본 발명에 따른 유기 분자 이외의 적어도 하나의, 즉 하나 이상의 이미터 및/또는 호스트 물질(들), 및 [0275] (c) 선택적으로 하나 이상의 염료 및/또는 하나 이상의 용매 [0276] 를 포함하거나 또는 구성된 조성물에 관한 것이다.4.발명은, 본 발명에 따른 유기 분자 또는 본 발명에 따른 조성물을 포함하는 유기 광전자 디바이스에 관한 것이다.5.발명의 일 구현예는 1000 cd/m2에서 5% 초과, 특히 8% 초과, 특히 10% 초과, 13% 초과, 16% 초과, 특히 20% 초과의 외부 양자 효율(EQE), 및/또는 420 nm 내지 500 nm, 특히 430 nm 내지 490 nm, 또는 440 nm 

### Tokenizing the Dataset

In [4]:
tokenizer_path = 'c09k_tokenizer_2'
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_path, vocab_size=8000,
                                              local_files_only=True, lowercase=False, strip_accents=False)
# tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


In [5]:
def encode_with_truncation(examples):
    """Mapping function to tokenize the sentences passed with truncation"""
    return tokenizer(examples["text"], truncation=True, padding="max_length",
                                     max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
    """Mapping function to tokenize the sentences passed without truncation"""
    return tokenizer(examples["text"], return_special_tokens_mask=True)

In [6]:
truncate_longer_samples = True

In [7]:
# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

In [8]:
# tokenizing the train dataset
max_length = 128
train_dataset = d['train'].map(encode, batched=True)
test_dataset = d['test'].map(encode, batched=True)
# d["train"], d["test"]

100%|██████████| 58/58 [00:03<00:00, 16.10ba/s]


In [9]:
# test_dataset
# Dataset({
#     features: ['text', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
#     num_rows: 77319
# })

In [10]:
# print(train_dataset[:2])
len(train_dataset)  # 715197

715197

In [11]:
if truncate_longer_samples:
    # remove other columns and set input_ids and attention_mask as PyTorch tensors
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
    # remove other columns, and remain them as Python lists
    test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
    train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

In [12]:
# for i in range(2):
#     print('input_text:', test_dataset['text'][i])
#     print('\ninput_ids:', test_dataset['input_ids'][i])
#     print('\ndecoded_input:', tokenizer.decode(test_dataset['input_ids'][i]))
#     print('-'*100, '\n')

### Loading the Model

In [13]:
# initialize the model with the config
vocab_size = tokenizer.vocab_size
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)

### Training

In [14]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [15]:
pre_trained_model_path = 'c09k_170k_pretrained'

In [17]:
# ?TrainingArguments

In [18]:
training_args = TrainingArguments(
#     model_name_or_path = pre_trained_model_path,
    output_dir=pre_trained_model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # 'steps': evaluate each `logging_steps`, 'epoch'  : each epoch
#     overwrite_output_dir=False,      
    resume_from_checkpoint = True,
    num_train_epochs=500.,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=64, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=8,  # evaluation batch size
    logging_steps=500,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=500,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [19]:
pre_trained_model_path

'c09k_170k_pretrained'

In [20]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train the model
# trainer.train('c09k_170k_pretrained/checkpoint-11000')
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, special_tokens_mask. If text, special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 715197
  Num Epochs = 500
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 8
  Total optimization steps = 698000


Step,Training Loss,Validation Loss
500,6.457400,5.800545
1000,5.598800,5.249725
1500,5.152500,4.762882
2000,4.670200,4.146641


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, special_tokens_mask. If text, special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 57989
  Batch size = 8
Saving model checkpoint to c09k_170k_pretrained/checkpoint-500
Configuration saved in c09k_170k_pretrained/checkpoint-500/config.json
Model weights saved in c09k_170k_pretrained/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, special_tokens_mask. If text, special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 57989
  Batch size = 8
Saving model checkpoint to c09k_170k_pretrained/checkpoint-1000
Configuration saved in c09k_170k_pretra

### Resuming training
* https://huggingface.co/transformers/v4.5.1/examples.html
* You can resume training from a previous checkpoint like this:
    * Pass --output_dir previous_output_dir without --overwrite_output_dir to resume training from the latest checkpoint in output_dir (what you would use if the training was interrupted, for instance).
    * Pass --model_name_or_path path_to_a_specific_checkpoint to resume training from that checkpoint folder.
    * Should you want to turn an example into a notebook where you’d no longer have access to the command line, 🤗 Trainer supports resuming from a checkpoint via trainer.train(resume_from_checkpoint).
    * If resume_from_checkpoint is True it will look for the last checkpoint in the value of output_dir passed via TrainingArguments.
    * If resume_from_checkpoint is a path to a specific checkpoint it will use that saved checkpoint folder to resume the training from.

In [56]:
# model.save_pretrained(os.path.join(pre_trained_model_path, 'checkpoint-4320'))

Configuration saved in c09k_pretrained_bert/checkpoint-4320/config.json
Model weights saved in c09k_pretrained_bert/checkpoint-4320/pytorch_model.bin


In [ ]:
# Gradient Accumulation steps * Total optimization steps = 8 * 540 = 4320

In [ ]:
# model.save_pretrained(model_path)

### Additional Training the Model

In [28]:
# load the model checkpoint
model1 = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-7500"))
# load the tokenizer
# tokenizer = BertTokenizerFast.from_pretrained(model_path)
tokenizer1 = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)

loading configuration file c09k_pretrained_bert/checkpoint-7500/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert/checkpoint-4320",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 64,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8000
}

loading weights file c09k_pretrained_bert/checkpoint-7500/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at c09k_pretrained_

vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


In [29]:
fill_mask = pipeline("fill-mask", model=model1, tokenizer=tokenizer1)

In [30]:
# perform predictions
# 인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치
# 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다
examples = [
    "인광성 유기 금속 이리듐 착체, [MASK] 소자, 발광 장치, 전자 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 [MASK] 및 이를 포함하는 유기 발광 소자에 관한 것이다",
    "인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, [MASK] 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 [MASK] 발광 소자에 관한 것이다",
]
for example in examples:
    print(fill_mask(example))
    for prediction in fill_mask(example):
        print(f"{prediction['sequence']}, confidence: {prediction['score']}")
    print("="*50)

[{'score': 0.46306195855140686, 'token': 1193, 'token_str': '발광', 'sequence': '인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.13777363300323486, 'token': 737, 'token_str': '표시', 'sequence': '인광성 유기 금속 이리듐 착체, 표시 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.05654741823673248, 'token': 1125, 'token_str': '일렉트로크로믹', 'sequence': '인광성 유기 금속 이리듐 착체, 일렉트로크로믹 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.03776992857456207, 'token': 3316, 'token_str': '광기능', 'sequence': '인광성 유기 금속 이리듐 착체, 광기능 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.03028818406164646, 'token': 663, 'token_str': '전기변색', 'sequence': '인광성 유기 금속 이리듐 착체, 전기변색 소자, 발광 장치, 전자 기기, 및 조명 장치'}]
인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조며

In [78]:
# Num examples = 19400
# Num Epochs = 50
# Total optimization steps = 3750 = 750*50 = 
# 로드한 모델 추가학습
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # 'steps': evaluate each `logging_steps`, 'epoch'  : each epoch
    overwrite_output_dir=True,      
    num_train_epochs=100.,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=32, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=8,  # evaluation batch size
    logging_steps=1500,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1500,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=5,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

trainer = Trainer(
    model=model1,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset1,
    eval_dataset=test_dataset1,
)

trainer.train()

using `logging_steps` to initialize `eval_steps` to 1500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 19400
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 7500


Step,Training Loss,Validation Loss
1500,4.143100,3.477852
3000,3.073100,2.746345
4500,2.481400,2.393077
6000,2.170100,2.215955
7500,2.018300,2.143648


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert/checkpoint-1500
Configuration saved in c09k_pretrained_bert/checkpoint-1500/config.json
Model weights saved in c09k_pretrained_bert/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [c09k_pretrained_bert/checkpoint-2000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2047
  Batch size = 8
Saving model checkpoint to c09k_

TrainOutput(global_step=7500, training_loss=2.7772110026041665, metrics={'train_runtime': 12492.1984, 'train_samples_per_second': 155.297, 'train_steps_per_second': 0.6, 'total_flos': 6.38038107242496e+16, 'train_loss': 2.7772110026041665, 'epoch': 99.99})

In [80]:
# load the model checkpoint
model1 = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-7500"))
# load the tokenizer
# tokenizer = BertTokenizerFast.from_pretrained(model_path)
tokenizer1 = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)
fill_mask = pipeline("fill-mask", model=model1, tokenizer=tokenizer1)
# perform predictions
# 인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치
# 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다
examples = [
    "인광성 유기 금속 이리듐 착체, [MASK] 소자, 발광 장치, 전자 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 [MASK] 및 이를 포함하는 유기 발광 소자에 관한 것이다",
    "인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, [MASK] 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 [MASK] 발광 소자에 관한 것이다",
]
for example in examples:
    print(fill_mask(example))
    for prediction in fill_mask(example):
        print(f"{prediction['sequence']}, confidence: {prediction['score']}")
    print("="*50)

loading configuration file c09k_pretrained_bert/checkpoint-7500/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert/checkpoint-4320",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 64,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8000
}

loading weights file c09k_pretrained_bert/checkpoint-7500/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at c09k_pretrained_

vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json
[{'score': 0.46306195855140686, 'token': 1193, 'token_str': '발광', 'sequence': '인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.13777363300323486, 'token': 737, 'token_str': '표시', 'sequence': '인광성 유기 금속 이리듐 착체, 표시 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.05654741823673248, 'token': 1125, 'token_str': '일렉트로크로믹', 'sequence': '인광성 유기 금속 이리듐 착체, 일렉트로크로믹 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.03776992857456207, 'token': 3316, 'token_str': '광기능', 'sequence': '인광성 유기 금속 이리듐 착체, 광기능 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.03028818406164646, 'token': 663, 'token_str': '전기변색', 'sequence': '인과

## Prepare FineTuning

### model load

In [31]:
from transformers import AdamW

In [32]:
# ?BertPreTrainedModel

In [34]:
# load the model checkpoint
model1 = BertForSequenceClassification.from_pretrained(os.path.join(model_path, "checkpoint-7500"), return_dict=True, num_labels=18)
# load the tokenizer
tokenizer1 = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)
optimizer = AdamW(model1.parameters(), lr=1e-5)

loading configuration file c09k_pretrained_bert/checkpoint-7500/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert/checkpoint-4320",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LABEL_16": 16,


vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [35]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model1.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model1.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

### finetuning data prepare

In [36]:
from torch.nn import functional as F

In [48]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 9881
})

In [37]:
encoding = tokenizer1(train_dataset['text'][:16], return_tensors='pt', padding=True, truncation=True, max_length=64)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [50]:
train_data_df = pd.read_csv('data/train_C09K11_220715.txt', sep='\t')  # text와 라벨 파일
test_dataset_df = pd.read_csv('data/test_C09K11_220715.txt', sep='\t')
train_dataset = Dataset.from_pandas(train_data_df)  # Dataset 객체 생성
test_dataset = Dataset.from_pandas(test_dataset_df)
finetune_dataset = DatasetDict()  # DatasetDict 객체 생성
finetune_dataset['train'] = train_dataset
finetune_dataset['test'] = test_dataset
finetune_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9881
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5203
    })
})

In [61]:
train_data_df[:3]

,text,label
0,광활성 형광체 프로브 및 이를 이용한 암세포 검출방법,1
1,"본 발명은 광활성 형광체 검출방법에 관한 것으로서, 화학식 1로 표시되는 화합물...",1
2,하기 [화학식 1]로 표시되는 OPA 또는 TPA 구조체를 포함하는 활용한 광활성 ...,1


In [51]:
# from datasets import load_dataset
# imdb = load_dataset("imdb")
# # imdb
# # DatasetDict({
# #     train: Dataset({
# #         features: ['text', 'label'],
# #         num_rows: 25000
# #     })
# #     test: Dataset({
# #         features: ['text', 'label'],
# #         num_rows: 25000
# #     })
# #     unsupervised: Dataset({
# #         features: ['text', 'label'],
# #         num_rows: 50000
# #     })
# # })

In [62]:
# # 학습 가능 여부 확인
# import torch
# labels = torch.tensor(train_dataset['label'][:16]).unsqueeze(0)
# outputs = model1(input_ids, attention_mask=attention_mask, labels=labels)
# loss = outputs.loss
# loss.backward()
# optimizer.step()

# # # loss 지정 계산
# # outputs = model1(input_ids, attention_mask=attention_mask)
# # loss = F.cross_entropy(labels, outputs.logits)
# # loss.backward()
# # optimizer.step()

In [53]:
num_warmup_steps = 2
n_epochs = 5
num_train_steps = n_epochs + 1
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)

In [41]:
epoch_loss = 0
epoch_acc = 0
for epoch in range(n_epochs):
    scheduler.step()
    

In [144]:
# train_dataset['text'][:5]

In [112]:
# train_dataset1 = train_dataset.shuffle(100)
# test_dataset1 = test_dataset.shuffle(100)

In [114]:
# train_dataset1['text'][:5]

In [77]:
# 토크나이저 함수 객체 생성
def preprocess_function(examples):
    return tokenizer1(examples["text"], truncation=True, max_length=64, padding=True)
def preprocess_function1(examples):
    return examples['label']

In [78]:
tokenized_finetune_dataset = finetune_dataset.map(preprocess_function, batched=True)  
# DatasetDict의 'text'를 토크나이징, finetune_dataset에는 features: ['text', 'label']만 있었으나,
# tokenized_finetune_dataset에는 'input_ids', 'token_type_ids', 'attention_mask'가 추가됨

100%|██████████| 6/6 [00:00<00:00, 14.96ba/s]


In [79]:
tokenized_finetune_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9881
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5203
    })
})

In [80]:
train_dataset1

Dataset({
    features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 19200
})

In [81]:
# token_train_dataset = {}
# finetune_dataset['sentences'] = list(map(preprocess_function, train_dataset1))
# token_train_dataset['labels'] = list(map(preprocess_function1, train_dataset1))
# # test_dataset = data_test1['text'].map(encode, batched=True)

In [82]:
# ?DataCollatorWithPadding

In [83]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer1)

In [68]:
# token_train_dataset['labels'][:5]

In [69]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [70]:
# training_args = TrainingArguments(output_dir="c09k_finetuned_bert")
training_args = TrainingArguments(
    output_dir="c09k_finetuned_bert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [72]:
trainer1 = Trainer(
    model=model1,
    args=training_args,
    train_dataset=tokenized_finetune_dataset['train'],
    eval_dataset=tokenized_finetune_dataset['test'],
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer1,
    data_collator=data_collator,
)

In [73]:
trainer1.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9881
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3090


Step,Training Loss
500,2.262800
1000,1.815100
1500,1.622500
2000,1.500900
2500,1.429500
3000,1.346000


Saving model checkpoint to c09k_finetuned_bert/checkpoint-500
Configuration saved in c09k_finetuned_bert/checkpoint-500/config.json
Model weights saved in c09k_finetuned_bert/checkpoint-500/pytorch_model.bin
Saving model checkpoint to c09k_finetuned_bert/checkpoint-1000
Configuration saved in c09k_finetuned_bert/checkpoint-1000/config.json
Model weights saved in c09k_finetuned_bert/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to c09k_finetuned_bert/checkpoint-1500
Configuration saved in c09k_finetuned_bert/checkpoint-1500/config.json
Model weights saved in c09k_finetuned_bert/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to c09k_finetuned_bert/checkpoint-2000
Configuration saved in c09k_finetuned_bert/checkpoint-2000/config.json
Model weights saved in c09k_finetuned_bert/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to c09k_finetuned_bert/checkpoint-2500
Configuration saved in c09k_finetuned_bert/checkpoint-2500/config.json
Model weights saved in c0

TrainOutput(global_step=3090, training_loss=1.6539026920849451, metrics={'train_runtime': 405.526, 'train_samples_per_second': 121.829, 'train_steps_per_second': 7.62, 'total_flos': 1625108636839680.0, 'train_loss': 1.6539026920849451, 'epoch': 5.0})

In [111]:
# load the model checkpoint
model2 = BertForSequenceClassification.from_pretrained(os.path.join('c09k_finetuned_bert', "checkpoint-3000"),
                                                       return_dict=True, num_labels=18)
# load the tokenizer
tokenizer2 = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)
optimizer = AdamW(model1.parameters(), lr=1e-5)

loading configuration file c09k_finetuned_bert/checkpoint-3000/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert/checkpoint-7500",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LA

vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [112]:
# ?pipeline
text_classifier = pipeline('text-classification', model=model2, tokenizer=tokenizer2)

In [113]:
# test_dataset['text'][:5]
# test_dataset['label'][:5]

In [120]:
for i in range(10):
    print('pred: ', text_classifier(test_dataset['text'][i]), ', label: ', test_dataset['label'][i])

pred:  [{'label': 'LABEL_9', 'score': 0.1434182971715927}] , label:  0
pred:  [{'label': 'LABEL_9', 'score': 0.1434182971715927}] , label:  0


RuntimeError: The size of tensor a (106) must match the size of tensor b (64) at non-singleton dimension 1

0

In [94]:
test_outputs = model1(test_input_ids, attention_mask=test_attention_mask, labels=test_labels)

AttributeError: 'list' object has no attribute 'size'

In [6]:
train_dataset = Dataset.from_text('data/train_C09K11_220715.txt')
# test_dataset = Dataset.from_text('data/test_C09K11_220715.txt')

In [7]:
train_dataset.features

{'text': Value(dtype='string', id=None)}

In [14]:
# train_dataset1 = []
# for line in train_dataset['text'][1:]:
#     text, label = line.split('\t')
#     train_dataset1.append({'text': text, 'label': label})
# print(train_dataset1[:5])
# test_dataset1 = []
# for line in test_dataset['text'][1:]:
#     text, label = line.split('\t')
#     test_dataset1.append({'text': text, 'label': label})
# print(test_dataset1[:5])

In [34]:
token_train_dataset

{'sentences': [{'input_ids': [480, 1531, 3771, 1111, 1307, 412, 1272, 1117, 6280, 1086, 681, 2370, 1030], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
  {'input_ids': [672, 785, 480, 1531, 3771, 2370, 5279, 882, 2578, 16, 746, 2183, 1138, 758, 1323, 16, 1597, 1710, 524, 1653, 3055, 401, 1323, 2300, 792, 558, 4418, 792, 3952, 546, 453, 584, 809, 1327, 6167, 389, 1329, 6280, 1086, 681, 383, 2370, 5279, 882, 4478, 398, 1059, 18, 3890, 1214, 672, 1399, 2750, 16, 3574, 1379, 12, 45, 281, 2143, 305, 262, 13, 660, 3574, 543, 12, 3016, 2143, 305, 262, 13, 915, 2950, 1106, 828, 916, 688, 384, 3956, 1830, 1360, 2053, 932, 1147, 448, 480, 1531, 1658, 1710, 524, 1205, 3055, 401, 2361, 736, 6280, 398, 2318, 786, 1976, 405, 467, 2005, 683, 402, 582, 389, 1757, 423, 239, 440, 7117, 623, 2281, 6280, 4418, 383, 2318, 786, 485, 2572, 1306, 558, 12, 3880, 3513, 284, 3221, 3513, 5830, 13, 414, 2572, 6406, 558, 414, 4181, 558, 12, 30

In [31]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 9881
})

In [34]:
# model1.train

In [32]:
classifier01 = pipeline("text-classification", model=model1, tokenizer=tokenizer1)
# classifier02 = pipeline("zero-shot-classification", model=model1, tokenizer=tokenizer1)

The model 'BertForTokenClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPTNeoForSequenceClassification', 'GPTJForSequenceClassification', 'IBertForSequenceClassification', 'LayoutLMForSequenceClassification', 'LayoutLMv2ForSequenceClassification', 'LayoutLMv3ForSequenceClassificati

In [37]:
classifier01(['인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 발광 기기, 및 조명 장치'])

Disabling tokenizer parallelism, we're using DataLoader multithreading already


KeyError: 10

### 그 외 참고용 자료

In [ ]:
# 명선 책임의 문장 분리 코드(3글자 이후 마침표가 등장하는 경우에 간혹 분리되는 경우가 있다. 미사용)
# REG_SENT_KO=r'([ㄱ-ㅣ가-힣]+[.]|[\n]|[:;!?])'
# REG_SENT_EN=r'([a-zA-Z]+[.]\s|[\n]|[:;!?])'

# def split_sentence(doc, regex):
#     s = 0
#     for m in re.finditer(regex, doc):
#         sent = doc[s:m.end()].strip()
#         s = m.end()
#         if not sent:
#             continue
#         yield sent

#     if s < len(doc):
#         sent = doc[s:].strip()
#         if sent:
#             yield sent